# Trabalho final

* Jhony Luiz de Almeida
* Manoel Vanildo Bento

### Base Kickstarter
* Fonte: Kaggle
* Link: https://www.kaggle.com/kemical/kickstarter-projects/download
* Descricão: base de campanhas do Kickstarter, coletadas em 2018, com seus respectivos status (sucesso ou fracasso)


#### Colunas:
- ID: Contém o ID do projeto
- name : Título do projeto
- category: Subcategoria do projeto (conjunto de valores pré-determinados)
- main_category: Macro-categoria do projeto (conjunto de valores pré-determinados)
- currency: Moeda (local) usada para as doações ao projeto
- launched: Data de início da campanha
- deadline: Data em que se encerra a campanha
- goal: Valor-objetivo para a campanha ter sucesso
- pledged: Valor arrecadado pela campanha
- backers: Número de apoiadores do projeto (ou número estimado, para o caso de previsões)
- country: País da pessoa que está lançando a campanha
- usd pledged: valor da coluna pledged convertida para dólares americanos (conversão feita pelo próprio kickstarter)
- usd_pledged_real: valor da coluna pledged convertida para dólares americanos (conversão feita pela API Fixer.io)
- usd_goal_real: valor da coluna goal convertida para dólares americanos (conversão feita pela API Fixer.io)
- state: Status da campanha, pode assumir os valores: Sucesso, Fracasso, Cancelado, Em Andamento

#### Problema definido:
- Prever se a campanha terá ou não sucesso, dados os atributos conhecidos em seu lançamento, como categoria, paiz do lançador da campanha, valor almejado, etc (os registros com estado cancelado ou em andamento serão desconsiderados)

In [1]:
# Leitura do arquivo de campanhas mais recentes (2018)
import pandas as pd
import calendar
data = pd.read_csv("data/ks-projects-201801.csv")

In [4]:
# Visualizando primeiros registros
data.head(8)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
5,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,52375.0,50000.00
6,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21,1000.0,2014-12-01 18:30:44,1205.0,successful,16,US,1205.0,1205.0,1000.00
7,1000030581,Chaser Strips. Our Strips make Shots their B*tch!,Drinks,Food,USD,2016-03-17,25000.0,2016-02-01 20:05:12,453.0,failed,40,US,453.0,453.0,25000.00


In [5]:
# Criando nova coluna duração da campanha, a partir das datas de início e fim

from datetime import datetime

data['launched.datetime'] = pd.to_datetime(data['launched'])
data['deadline.datetime'] = pd.to_datetime(data['deadline'])
data['duration_days'] = (data['deadline.datetime'] - data['launched.datetime'])
data['duration_days'] = data['duration_days'].map(lambda x: int(x.days))
data.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,launched.datetime,deadline.datetime,duration_days
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95,2015-08-11 12:12:28,2015-10-09,58
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00,2017-09-02 04:43:57,2017-11-01,59
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00,2013-01-12 00:20:50,2013-02-26,44
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00,2012-03-17 03:24:11,2012-04-16,29
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00,2015-07-04 08:35:03,2015-08-29,55


In [6]:
# Removendo status que não serão considerados, e separando atributos das classes (state)

indexes_to_exclude = data[(data['state'] == 'undefined') | (data['state'] == 'suspended') | (data['state'] == 'live') | (data['state'] == 'canceled')].index
data.drop(indexes_to_exclude, inplace = True)
attributes = data.drop('state', axis=1)
classes = data['state']
classes.value_counts()

failed        197719
successful    133956
Name: state, dtype: int64

In [7]:
# Checando valores de moedas
data['currency'].value_counts()

USD    261511
GBP     29476
EUR     14378
CAD     12375
AUD      6621
SEK      1510
MXN      1411
NZD      1274
DKK       929
CHF       652
NOK       584
HKD       477
SGD       454
JPY        23
Name: currency, dtype: int64

In [8]:
# Checando valores de países
data['country'].value_counts()

US      261360
GB       29454
CA       12370
AU        6616
DE        3436
FR        2520
NL        2411
IT        2369
ES        1873
SE        1509
MX        1411
NZ        1274
DK         926
IE         683
CH         652
NO         582
BE         523
AT         485
HK         477
SG         454
N,0"       210
LU          57
JP          23
Name: country, dtype: int64

In [9]:
# Checando valores de macro-categorias
data['main_category'].value_counts()

Film & Video    56527
Music           45949
Publishing      35445
Games           28521
Technology      27050
Art             25641
Design          25364
Food            22054
Fashion         19775
Theater         10242
Comics           9878
Photography      9689
Crafts           7818
Journalism       4149
Dance            3573
Name: main_category, dtype: int64

In [10]:
# Checando valores de subcategorias
# data['category'].value_counts()

In [11]:
# Removendo atributos que não serão usados na classificação: datas, valor arrecadado, valor objetivo em dólar, id e título do projeto
attributes.drop(['category', 'ID', 'name', 'deadline', 'launched', 'launched.datetime', 'deadline.datetime', 'pledged', 'usd pledged', 'usd_pledged_real', 'usd_goal_real'], axis=1, inplace = True)

In [13]:
# Conversão dos atributos de categorias com one hot encoder e os numéricos com MinMaxScaler

from category_encoders import one_hot
from sklearn import preprocessing

encoder = one_hot.OneHotEncoder(cols = ['main_category', 'country', 'currency'])
goal_scaler = preprocessing.MinMaxScaler()
backers_scaler = preprocessing.MinMaxScaler()
duration_scaler = preprocessing.MinMaxScaler()

new_attributes = encoder.fit_transform(attributes)
new_attributes['goal'] = goal_scaler.fit_transform(new_attributes['goal'].values.reshape(-1, 1))
new_attributes['backers'] = backers_scaler.fit_transform(new_attributes['backers'].values.reshape(-1, 1))
new_attributes['duration_days'] = duration_scaler.fit_transform(new_attributes['duration_days'].values.reshape(-1, 1))

new_attributes.head()

,main_category_1,main_category_2,main_category_3,main_category_4,main_category_5,main_category_6,main_category_7,main_category_8,main_category_9,main_category_10,...,country_15,country_16,country_17,country_18,country_19,country_20,country_21,country_22,country_23,duration_days
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.637363
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.648352
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.483516
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.318681
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.373626


In [14]:
# Criando classificadores candidatos

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

classifiers = {
    'Logistic Regression': LogisticRegression(solver = 'liblinear', max_iter = 100),
    'Tree Classifier': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(n_estimators = 100)
}

In [15]:
# Treinando os classificadores, separando 20% da base para teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(new_attributes, classes, test_size=0.20)

for name, classifier in classifiers.items():
    classifier.fit(X_train, y_train)

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [17]:
# Avaliação dos modelos, e selecionando o com melhor Kappa para ser usado no sensor
top_classifier = None
top_name = None
top_kappa = 0
for name, classifier in classifiers.items():
    y_pred = classifier.predict(X_test)
    
    print("\n----- Resultado da Avaliação do Modelo {} -----".format(name))
    
    scores = cross_val_score(classifier, X_test, y_test, cv=5)
    print("\nScores de cross validation: ", scores)

    print("\nMatriz de confusão:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    
    print(classification_report(y_test, y_pred))
    
    accuracy = cm[1][1] / (cm[1][1] + cm[0][1])
    sensitivity = cm[1][1] / (cm[1][1] + cm[1][0])
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])
    total = np.sum(cm)
    observed_accuracy = (cm[0][0] + cm[1][1]) / total
    expected_accuracy = (((cm[0][0] + cm[1][0]) * (cm[0][0] + cm[0][1]) / total) + ((cm[0][1] + cm[1][1]) * (cm[1][0] + cm[1][1]) / total)) / total
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy)
    print("Acurácia: {}".format(accuracy))
    print("Sensibilidade: {}".format(sensitivity))
    print("Especificidade: {}".format(specificity))
    print("Kappa: {}".format(kappa))

    if kappa > top_kappa:
        top_kappa = kappa
        top_name = name
        top_classifier = classifier
        
print("\nClassificador escolhido: ", top_name)


----- Resultado da Avaliação do Modelo Logistic Regression -----

Scores de cross validation:  [0.63687067 0.63536328 0.63209467 0.64043419 0.63538369]

Matriz de confusão:
[[34025  5447]
 [18172  8691]]
              precision    recall  f1-score   support

      failed       0.65      0.86      0.74     39472
  successful       0.61      0.32      0.42     26863

    accuracy                           0.64     66335
   macro avg       0.63      0.59      0.58     66335
weighted avg       0.64      0.64      0.61     66335

Acurácia: 0.6147262696279531
Sensibilidade: 0.32353050664482746
Especificidade: 0.8620034454803405
Kappa: 0.20072040400204497

----- Resultado da Avaliação do Modelo Tree Classifier -----

Scores de cross validation:  [0.90171842 0.89448297 0.90171101 0.90034675 0.89808533]

Matriz de confusão:
[[36519  2953]
 [ 3450 23413]]
              precision    recall  f1-score   support

      failed       0.91      0.93      0.92     39472
  successful       0.89      0.8

In [18]:
# Salvando o modelo escolhido e os encoders/scalers para uso na aplicação do sensor
import joblib
joblib.dump(top_classifier, 'models/kickstarter_classifier.joblib')
joblib.dump(encoder, 'models/kickstarter_encoder.joblib')
joblib.dump(goal_scaler, 'models/kickstarter_goal_scaler.joblib')
joblib.dump(backers_scaler, 'models/kickstarter_backers_scaler.joblib')
joblib.dump(duration_scaler, 'models/kickstarter_duration_scaler.joblib')

['models/kickstarter_duration_scaler.joblib']

In [19]:
top_classifier.predict_proba(X_test)

array([[1.00000000e+00, 0.00000000e+00],
       [3.52941176e-05, 9.99964706e-01],
       [0.00000000e+00, 1.00000000e+00],
       ...,
       [2.90000000e-02, 9.71000000e-01],
       [3.97085533e-01, 6.02914467e-01],
       [9.80000000e-01, 2.00000000e-02]])